<a href="https://colab.research.google.com/github/SANJAY-VIRAT/CRICKET-STATS-WEBSITE/blob/main/HACKATHON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
#importing required libraries
import pandas as pd
import json
import sqlite3

In [64]:
#loading orders data from csv file
orders_df=pd.read_csv("orders.csv")
print("Orders shape:",orders_df.shape)
orders_df.head()

Orders shape: (10000, 6)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [65]:
#loading users data from json file by convertion it to dataframe
with open("users.json","r") as file:
  users_json=json.load(file)
users_df=pd.DataFrame(users_json)
print("Users shape:",users_df.shape)
users_df.head()

Users shape: (3000, 4)


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [66]:
#loading restaurants table into dataframe by creating temporary memory
connection=sqlite3.connect("memory:")
with open("restaurants.sql","r") as file:
  sql_commands=file.read()
connection.executescript("DROP TABLE IF EXISTS restaurants;" + sql_commands)
restaurants_df=pd.read_sql_query("SELECT * FROM restaurants",connection)
print("Restaurants Shape:",restaurants_df.shape)
restaurants_df.head()

Restaurants Shape: (500, 4)


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [67]:
#joining orders with users using user_id
merged_df=pd.merge(orders_df,users_df,on="user_id",how="left")
#joining restaurants using restaurants_id
merged_df=pd.merge(merged_df,restaurants_df,on="restaurant_id",how="left")
print("Final merged shape:",merged_df.shape)
merged_df.head()

Final merged shape: (10000, 12)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [68]:
#saving the file
merged_df.to_csv("final_food_delivery_dataset.csv",index=False)
print("final_food_delivery_dataset.csv created_successfully!")

final_food_delivery_dataset.csv created_successfully!


In [69]:
#1
gold=merged_df[merged_df["membership"]=="Gold"]
gold.groupby("city")["total_amount"].sum().sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [70]:
#2
merged_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [71]:
#3
user_totals=merged_df.groupby("user_id")["total_amount"].sum()
count=(user_totals>1000).sum()
print("Users:",count)

Users: 2544


In [72]:
#4
bins=[3.0,3.5,4.0,4.5,5.0]
labels=["3.0-3.5","3.6-4.0","4.1=4.5","4.6-5.0"]
merged_df["rating_range"]=pd.cut(merged_df["rating"],bins=bins,labels=labels)
merged_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)

/tmp/ipython-input-3215550619.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6-5.0,2197030.75
4.1=4.5,1960326.26
3.0-3.5,1881754.57
3.6-4.0,1717494.41


In [73]:
#5
gold.groupby("city")["total_amount"].mean().sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [74]:
#6
restaurants_per_cuisine=merged_df.groupby("cuisine")["restaurant_id"].nunique()
revenue_per_cuisine=merged_df.groupby("cuisine")["total_amount"].sum()
print("Restaurants:\n",restaurants_per_cuisine.sort_values())
print("\nRevenue:\n",revenue_per_cuisine.sort_values(ascending=False))

Restaurants:
 cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

Revenue:
 cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64


In [75]:
#7
gold_ordwes=len(gold)
total_orders=len(merged_df)
percent=round((gold_ordwes/total_orders)*100)
print(percent,"%")

50 %


In [76]:
#8
stats=merged_df.groupby("restaurant_name_y").agg({"total_amount":"mean","restaurant_id":"count"})
filtered=stats[stats["restaurant_id"]<20]
filtered.sort_values("total_amount",ascending=False)

,total_amount,restaurant_id
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16
...,...,...
Restaurant_184,621.828947,19
Restaurant_498,596.815556,18
Restaurant_192,589.972857,14


In [77]:
#9
merged_df.groupby(["membership","cuisine"])["total_amount"].sum().sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [78]:
#10
merged_df["order_date"]=pd.to_datetime(merged_df["order_date"])
merged_df["quarter"]=merged_df["order_date"].dt.quarter
merged_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)

/tmp/ipython-input-2653835382.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  merged_df["order_date"]=pd.to_datetime(merged_df["order_date"])


,total_amount
quarter,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [79]:
merged_df.groupby("restaurant_name_y").agg({
    "total_amount":"mean",
    "restaurant_id":"count"
}).query("restaurant_id < 20").sort_values("total_amount", ascending=False).head()


,total_amount,restaurant_id
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16


In [80]:
#11
gold_orders=len(merged_df[merged_df["membership"]=="Gold"])
gold_orders

4987

In [81]:
#12
hyd_revenue=merged_df[merged_df["city"]=="Hyderabad"]["total_amount"].sum()
round(hyd_revenue)

1889367

In [82]:
#13
merged_df["user_id"].nunique()

2883

In [83]:
#14
gold_avg=merged_df[merged_df["membership"]=="Gold"]["total_amount"].mean()
round(gold_avg,2)

np.float64(797.15)

In [84]:
#15
highest_rating_orders=len(merged_df[merged_df["rating"]>=4.5])
highest_rating_orders

3374

In [85]:
#16
gold=merged_df[merged_df["membership"]=="Gold"]
top_city=gold.groupby("city")["total_amount"].sum().idxmax()
top_city
len(gold[gold["city"]==top_city])

1337

In [86]:
#17
pd.merge(orders_df,users_df,on="user_id")

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold
